In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

## Memory with RAG

We will use selenium to scrape a website that requires javascript

In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# Chrome driver path
chromedriver_path = "/Users/timothylee/Desktop/chromedriver"

# Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")

service = Service(executable_path=chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get("https://sguniguide.tech/")
time.sleep(5)

# Get the innerHTML of the body
docs = driver.find_element(by="tag name", value="body").get_attribute('innerHTML')
driver.quit()

In [148]:
from langchain_openai import OpenAIEmbeddings
llm = ChatOpenAI()
embeddings = OpenAIEmbeddings()

In [47]:
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import (
    Language,
    RecursiveCharacterTextSplitter,
)

# We will split text into chunks using HTML language
text_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.HTML, chunk_size=500, chunk_overlap=0)
documents = text_splitter.create_documents([docs])

# save collection to disk
vector = Chroma.from_documents(documents, embeddings, persist_directory="./chroma_db")

In [52]:
# delete the collection
# vector.delete_collection()

# load the collection from disk
vector = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

# get the number of documents in the collection
vector._collection.count()

82

In [38]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
                                          <context>
                                          {context}
                                          </context>
                                          Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [53]:
from langchain.chains import create_retrieval_chain

# Retrieve k relevant documents
retriever = vector.as_retriever(search_kwargs={"k": 3})
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [56]:
response = retrieval_chain.invoke({"input": "What schools are in this website?"})
print(response['answer'])

The website mentioned in the context is Singapore University of Technology and Design.


In [186]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory, ConversationBufferWindowMemory, ConversationSummaryBufferMemory, ConversationTokenBufferMemory
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter
from langchain_core.messages import AIMessage, HumanMessage

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Correct and grammatically complete sentences is required. Only return the query."),
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

## ConversationBufferMemory

The ConversationBufferMemory is the most straightforward conversational memory in LangChain. As we described above, the raw input of the past conversation between the human and AI is passed — in its raw form — to the {history} parameter.

In [116]:
chat_history = [
    HumanMessage(content="Hello what schools are mentioned?"),
    AIMessage(content="The website mentioned in the context is Singapore University of Technology and Design.")
]

memory = ConversationBufferMemory(return_messages=True)
memory.chat_memory.add_messages(chat_history)

In [117]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hello what schools are mentioned?'),
  AIMessage(content='The website mentioned in the context is Singapore University of Technology and Design.')]}

In [118]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the following question based only on the provided context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

final_chain = (
    RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | retrieval_chain
)

input = {"input": "What other schools are mentioned?"}
response = final_chain.invoke(input)
print(response['answer'])
memory.save_context(input, {"output": response['answer']})

The other schools mentioned in the context are Singapore Institute of Technology and Singapore University of Social Sciences.


In [119]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hello what schools are mentioned?'),
  AIMessage(content='The website mentioned in the context is Singapore University of Technology and Design.'),
  HumanMessage(content='What other schools are mentioned?'),
  AIMessage(content='The other schools mentioned in the context are Singapore Institute of Technology and Singapore University of Social Sciences.')]}

In [120]:
response = final_chain.invoke({"input": "Describe the accountancy course at SMU?"})
print(response['answer'])

The accountancy course at SMU provides a holistic education that is broad-based with a strong accounting core. It covers areas such as business, technology, entrepreneurship, leadership skills, communications, and social responsibility. Additionally, all students are guaranteed a Second Major, which they can choose from any school across SMU.


## ConversationBufferWindowMemory
The ConversationBufferWindowMemory acts in the same way as our earlier “buffer memory” but adds a window to the memory. Meaning that we only keep a given number of past interactions before “forgetting” them.

In [127]:
memory = ConversationBufferWindowMemory(return_messages=True, k=2)
memory.chat_memory.add_messages(chat_history)
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hello what schools are mentioned?'),
  AIMessage(content='The website mentioned in the context is Singapore University of Technology and Design.')]}

In [128]:
input = {"input": "What other schools are mentioned?"}

final_chain = (
    RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | retrieval_chain
)

response = final_chain.invoke(input)
print(response['answer'])
memory.save_context(input, {"output": response['answer']})

The other schools mentioned are Singapore Institute of Technology and Singapore University of Social Sciences.


In [129]:
input = {"input": "Tell me more about Singapore Institute of Technology?"}
response = final_chain.invoke(input)
print(response['answer'])
memory.save_context(input, {"output": response['answer']})

Singapore Institute of Technology (SIT) is a university in Singapore that offers applied degree programs in partnership with reputable overseas universities. SIT focuses on applied learning and aims to produce graduates who are ready for the workforce. The university collaborates closely with industry partners to ensure that its curriculum is relevant and up-to-date. SIT has a strong emphasis on hands-on learning through internships, projects, and industry attachments.


In [130]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='What other schools are mentioned?'),
  AIMessage(content='The other schools mentioned are Singapore Institute of Technology and Singapore University of Social Sciences.'),
  HumanMessage(content='Tell me more about Singapore Institute of Technology?'),
  AIMessage(content='Singapore Institute of Technology (SIT) is a university in Singapore that offers applied degree programs in partnership with reputable overseas universities. SIT focuses on applied learning and aims to produce graduates who are ready for the workforce. The university collaborates closely with industry partners to ensure that its curriculum is relevant and up-to-date. SIT has a strong emphasis on hands-on learning through internships, projects, and industry attachments.')]}

In [131]:
# Original first question is 'Hello what schools are mentioned?'
input = {"input": "What was my first question?"}
response = final_chain.invoke(input)
print(response['answer'])
memory.save_context(input, {"output": response['answer']})

Your first question was: "What other schools are mentioned?"


In [132]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Tell me more about Singapore Institute of Technology?'),
  AIMessage(content='Singapore Institute of Technology (SIT) is a university in Singapore that offers applied degree programs in partnership with reputable overseas universities. SIT focuses on applied learning and aims to produce graduates who are ready for the workforce. The university collaborates closely with industry partners to ensure that its curriculum is relevant and up-to-date. SIT has a strong emphasis on hands-on learning through internships, projects, and industry attachments.'),
  HumanMessage(content='What was my first question?'),
  AIMessage(content='Your first question was: "What other schools are mentioned?"')]}

## ConversationSummaryMemory

Using ConversationBufferMemory, we very quickly use a lot of tokens and even exceed the context window limit of even the most advanced LLMs available today.

To avoid excessive token usage, we can use ConversationSummaryMemory. As the name would suggest, this form of memory summarizes the conversation history before it is passed to the {history} parameter.

In [152]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_messages(chat_history)
print(history)

memory = ConversationSummaryMemory.from_messages(llm=llm, chat_memory=history, return_messages=True)
memory.load_memory_variables({})

Human: Hello what schools are mentioned?
AI: The website mentioned in the context is Singapore University of Technology and Design.


{'history': [SystemMessage(content='The human asks what schools are mentioned. The AI responds that the website mentioned in the context is Singapore University of Technology and Design.')]}

In [153]:
input = {"input": "I am Timothy and I am interested in the courses offered by the universities. Can you help me?"}

final_chain = (
    RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | retrieval_chain
)

response = final_chain.invoke(input)
print(response['answer'])
memory.save_context(input, {"output": response['answer']})

I'm sorry, but based on the provided context, I can only confirm that the Singapore University of Technology and Design is mentioned. I do not have information about the courses offered by the universities mentioned in the context.


In [154]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human asks what schools are mentioned and the AI responds that the website mentioned in the context is Singapore University of Technology and Design. When Timothy expresses interest in the courses offered by universities, the AI apologizes for not having information about courses offered and can only confirm the mention of Singapore University of Technology and Design.')]}

In [155]:
response = final_chain.invoke({"input": "What's my name?"})
print(response['answer'])

Your name is Timothy.


## ConversationSummaryBufferMemory

The ConversationSummaryBufferMemory is a mix of the ConversationSummaryMemory and the ConversationBufferWindowMemory. It summarizes the earliest interactions in a conversation while maintaining the max_token_limit most recent tokens in their conversation.

In [180]:
memory = ConversationSummaryBufferMemory(llm=llm, return_messages=True, max_token_limit=40)
memory.chat_memory.add_messages(chat_history)
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hello what schools are mentioned?'),
  AIMessage(content='The website mentioned in the context is Singapore University of Technology and Design.')]}

In [181]:
input = {"input": "I am Timothy and I am interested in SMU. Can you help me?"}

final_chain = (
    RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | retrieval_chain
)

response = final_chain.invoke(input)
print(response['answer'])
memory.save_context(input, {"output": response['answer']})

I'm sorry, but based on the context provided, the website mentioned is Singapore University of Technology and Design, not Singapore Management University (SMU). If you have any questions specifically about Singapore University of Technology and Design, I'd be happy to help.


In [182]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human inquires about the schools mentioned in the context. The AI responds that the website referred to is Singapore University of Technology and Design. Timothy expresses interest in SMU and seeks help. The AI clarifies that based on the context provided, the website mentioned is Singapore University of Technology and Design, not SMU, offering assistance with questions related to the former.')]}

In [184]:
reponse = final_chain.invoke({"input": "What was I asking for?"})
print(reponse['answer'])
memory.save_context({"input": "What was I asking for?"}, {"output": response['answer']})

Based on the context provided, you were asking about the schools mentioned, specifically Singapore University of Technology and Design.


In [185]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human inquires about the schools mentioned in the context, and the AI responds that the website referred to is Singapore University of Technology and Design. Timothy expresses interest in SMU and seeks help. The AI clarifies that based on the context provided, the website mentioned is Singapore University of Technology and Design, not SMU, offering assistance with questions related to the former. When the human asks about what was being inquired, the AI reiterates that the website mentioned is Singapore University of Technology and Design, not Singapore Management University (SMU), and offers help for questions related to the former.')]}

## ConversationTokenBufferMemory

ConversationTokenBufferMemory keeps a buffer of recent interactions in memory, and uses token length rather than number of interactions to determine when to flush interactions.

In [190]:
memory = ConversationTokenBufferMemory(llm=llm, return_messages=True, max_token_limit=50)
memory.chat_memory.add_messages(chat_history)
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hello what schools are mentioned?'),
  AIMessage(content='The website mentioned in the context is Singapore University of Technology and Design.')]}

In [196]:
input = {"input": "I am Timothy and I am interested in NTU. Can you help me?"}

final_chain = (
    RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | retrieval_chain
)

response = final_chain.invoke(input)
print(response['answer'])
memory.save_context(input, {"output": response['answer']})

Based on the provided context, it seems that you are interested in Nanyang Technological University (NTU). If you have any specific questions about NTU or need assistance with something related to the university, feel free to ask for help.Based on the provided context, it seems that you are interested in Nanyang Technological University (NTU).


In [192]:
# Notice the memory saves up to 50 tokens
memory.load_memory_variables({})

{'history': [AIMessage(content='Yes, I can help you with information about Nanyang Technological University (NTU). What specific information are you looking for?')]}

In [197]:
# streaming output for better ux
for chunk in final_chain.stream(input):
    if 'answer' in chunk:
        print(chunk['answer'], end="", flush=True)

Yes, I can help you with information about Nanyang Technological University (NTU). What specific information are you looking for?